# ✅ FAST MODE TOGGLE
FAST_QA_MODE = True

# -----------------------------
# 🔁 Quantum-Inspired Mutation
# -----------------------------
import numpy as np
import copy

def quantum_mutate(model_dict, temperature=1.0):
    """
    Apply a small, random mutation to model hyperparameters.
    Accept worse changes with some probability (simulated tunneling).
    """
    model_copy = copy.deepcopy(model_dict)
    params = model_copy["params"]

    # Example mutation: tweak alpha for ElasticNet
    if "alpha" in params:
        jump = np.random.normal(0, 0.05)  # small change
        old_alpha = params["alpha"]
        params["alpha"] = max(0.0001, old_alpha + jump)

    # Add more hyperparam types as needed (learning_rate, C, etc)
    model_copy["model"] = make_child_model(params)  # rebuild with new params
    return model_copy

# -----------------------------
# ⚡ Parallel Evaluation
# -----------------------------
from concurrent.futures import ThreadPoolExecutor

def evaluate_single_model(model_dict, resampled_datasets, X_test, y_test):
    model_dict = train_and_score_model(
        model_dict, resampled_datasets, X_test, y_test, debug=False
    )
    return model_dict

def parallel_evaluate_population(model_population, resampled_datasets, X_test, y_test, workers=4):
    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = [
            executor.submit(evaluate_single_model, model, resampled_datasets, X_test, y_test)
            for model in model_population
        ]
        return [f.result() for f in futures]

# -----------------------------
# 🧬 Fast Quantum Battle Mode
# -----------------------------
def breed_and_battle_with_population_fast(
    model_population,
    resampled_datasets,
    X_test,
    y_test,
    generations=3,
    top_k=3,
    dataset_name=None,
    baseline_pr_auc=None,
    debug=False,
    temperature_init=1.0,
    cooling_rate=0.8,
):
    temperature = temperature_init

    for gen in range(generations):
        print(f"\n🌀 Generation {gen + 1} | Temp: {temperature:.4f}")

        # 🌌 Quantum-inspired mutation
        mutated_models = [quantum_mutate(m, temperature) for m in model_population]

        # ⚡ Evaluate all models in parallel
        evaluated_models = parallel_evaluate_population(
            mutated_models, resampled_datasets, X_test, y_test
        )

        # 🏆 Select top_k survivors
        sorted_models = sorted(evaluated_models, key=lambda x: -x["pr_auc"])
        model_population = sorted_models[:top_k]

        if debug:
            for m in model_population:
                print(f"  ✅ {m['label']}: PR AUC = {m['pr_auc']:.4f}")

        # ❄️ Cool temperature
        temperature *= cooling_rate

    return model_population
